<a href="https://colab.research.google.com/github/ml-group8/assignment/blob/main/Stock_Price_Prediction_with_Sentiment_Analysis_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import json
import ast
import yaml
import urllib
from datetime import datetime, timedelta
from google.colab import drive
import re

!pip install flair
import flair

In [ ]:
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

In [ ]:
# TODO: There should be a better way to do this , existing library ??
# Load tweets we extracted from running the previous notebook and had stored on github

# TODO: Tweets may contain re-tweets - can we remove with suggestion from 
#   https://twittercommunity.com/t/how-do-i-get-full-tweet-text-in-v2/142129/3
df_tweets = pd.read_csv('https://raw.githubusercontent.com/ml-group8/assignment/main/elec_vehicle_tweets.csv')
df_tweets.drop(df_tweets.columns[0], axis=1, inplace=True)
df_tweets.drop(df_tweets.columns[3], axis=1, inplace=True)
df_tweets.head()



,created_at,id,text
0,2021-03-20T15:59:46.000Z,1373303259964788740,@QuestMalloy I agree. I was looking to buy an ...
1,2021-03-20T15:59:30.000Z,1373303194202353670,RT @JessiSheron: this line read is so funny!??...
2,2021-03-20T15:59:09.000Z,1373303103202725897,RT @JessiSheron: this line read is so funny!??...
3,2021-03-20T15:59:03.000Z,1373303077697126401,@wef Germany electric car and 60% US states ma...
4,2021-03-20T15:58:49.000Z,1373303019287248902,@Greenpeace Germany electric car and 60% US st...


In [ ]:
drive.mount('/content/gdrive', force_remount=True)

In [25]:
cd /content/gdrive/MyDrive

/content/gdrive/MyDrive


In [33]:
# Process the tweets 
whitespace = re.compile(r"\s+")
web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
tesla = re.compile(r"(?i)@Tesla(?=\b)")
user = re.compile(r"(?i)@[a-z0-9_]+")

def clean_tweet(tweet):
  tweet = whitespace.sub(' ', tweet)
  tweet = web_address.sub('', tweet)
  tweet = tesla.sub('Tesla', tweet)
  tweet = user.sub('', tweet)
  return tweet

def predict_sentiment(tweet):
  type(tweet)
  sentence = flair.data.Sentence(tweet)
  sentiment_model.predict(sentence)
  label = sentence.labels[0]
  return (label.value, label.score)

def assign_sentiment_and_score(df):
  for idx, row in df.iterrows():
    sentiment_analysis = predict_sentiment(df.at[idx,'tweet'])
    df.at[idx,'sentiment'] = sentiment_analysis[0]
    df.at[idx,'confidence'] = sentiment_analysis[1]

df_tweets['tweet'] = df_tweets['text'].map(lambda text: clean_tweet(text))

print('Sentiment analysis starting at : ' + str(datetime.now()))
assign_sentiment_and_score(df_tweets)
df_tweets.to_csv('df_tweets_sentiment.csv')
print('Sentiment analysis completed at : ' + str(datetime.now()))



Sentiment analysis starting at : 2021-03-21 16:15:28.185288
Sentiment analysis completed at : 2021-03-21 16:37:04.642169
